In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.signal import detrend
import os
import pickle

# ==============================
# Step 1: Load and Preprocess Data
# ==============================
def load_and_preprocess_data(file_path, target_column, sequence_length, scaler_type='StandardScaler', detrend_data=False):
    # Load data
    data = pd.read_csv(file_path)

    # Detrend the data if required (e.g., for Battery Level)
    if detrend_data:
        data[target_column] = detrend(data[target_column])

    # Normalize the target column
    if scaler_type == 'MinMaxScaler':
        scaler = MinMaxScaler()
    else:
        scaler = StandardScaler()

    data[target_column] = scaler.fit_transform(data[[target_column]])

    # Create sequences
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[target_column].iloc[i:i + sequence_length].values
        sequences.append(seq)

    sequences = np.array(sequences)
    sequences = np.expand_dims(sequences, axis=2)  # Add feature dimension
    return sequences, scaler

# ==============================
# Step 2: Define LSTM Autoencoder
# ==============================
def create_lstm_autoencoder(sequence_length, input_dim, latent_dim=32):
    # Encoder
    input_layer = Input(shape=(sequence_length, input_dim))
    encoded = LSTM(latent_dim, activation='relu', return_sequences=False)(input_layer)
    encoded = Dense(latent_dim // 2, activation='relu')(encoded)

    # Decoder
    decoded = RepeatVector(sequence_length)(encoded)
    decoded = LSTM(latent_dim, activation='relu', return_sequences=True)(decoded)
    decoded = TimeDistributed(Dense(input_dim))(decoded)

    # Autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mae')
    return autoencoder

# ==============================
# Step 3: Train and Save Models
# ==============================
def train_and_save_model(sequences, model_path, sequence_length, input_dim, latent_dim=32, epochs=50, batch_size=32):
    # Define LSTM Autoencoder
    model = create_lstm_autoencoder(sequence_length, input_dim, latent_dim)

    # Train model with early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(sequences, sequences, epochs=epochs, batch_size=batch_size, validation_split=0.2, shuffle=True, callbacks=[early_stopping])

    # Save model
    model.save(model_path)
    print(f"Model saved to {model_path}")
    return model

# ==============================
# Step 4: Main Script
# ==============================
if __name__ == "__main__":
    # Parameters
    file_path = "C:/Users/saura/OneDrive/Desktop/data_streaming/data_generators/chennai/dwlrdata_chennai_anomalies_final(1).csv"
    sequence_length = 30         # Number of timesteps per sequence
    epochs = 50                  # Training epochs
    batch_size = 32              # Batch size
    output_dir = "C:/Users/saura/Downloads/trained_model_v2"  # Directory to save models

    # Create output directory if not exists
    os.makedirs(output_dir, exist_ok=True)

    # Depth
    print("Training model for Depth...")
    depth_sequences, depth_scaler = load_and_preprocess_data(
        file_path, target_column="Depth", sequence_length=sequence_length, scaler_type='StandardScaler'
    )
    train_and_save_model(depth_sequences, os.path.join(output_dir, "depth_model_chennai.h5"), sequence_length, input_dim=1, latent_dim=32, epochs=epochs, batch_size=batch_size)

    # Temperature
    print("Training model for Temperature...")
    temperature_sequences, temperature_scaler = load_and_preprocess_data(
        file_path, target_column="Temperature", sequence_length=sequence_length, scaler_type='StandardScaler'
    )
    train_and_save_model(temperature_sequences, os.path.join(output_dir, "temperature_model_chennai.h5"), sequence_length, input_dim=1, latent_dim=32, epochs=epochs, batch_size=batch_size)

    # Battery Level
    print("Training model for Battery Level...")
    battery_sequences, battery_scaler = load_and_preprocess_data(
        file_path, target_column="Battery Level", sequence_length=sequence_length, scaler_type='MinMaxScaler', detrend_data=True
    )
    train_and_save_model(battery_sequences, os.path.join(output_dir, "battery_mode_chennail.h5"), sequence_length, input_dim=1, latent_dim=64, epochs=epochs, batch_size=batch_size)

    # Save scalers for future use
    scalers = {
        "Depth": depth_scaler,
        "Temperature": temperature_scaler,
        "Battery Level": battery_scaler
    }
    scaler_path = os.path.join(output_dir, "scalers_chennai.pkl")
    with open(scaler_path, "wb") as f:
        pickle.dump(scalers, f)
    print(f"Scalers saved to {scaler_path}")

    print("Training complete. Models and scalers are saved.")


Training model for Depth...
Epoch 1/50
730/730 [==============================] - 14s 16ms/step - loss: 0.0980 - val_loss: 0.0596
Epoch 2/50
730/730 [==============================] - 11s 16ms/step - loss: 0.0458 - val_loss: 0.0401
Epoch 3/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0401 - val_loss: 0.0376
Epoch 4/50
730/730 [==============================] - 10s 14ms/step - loss: 0.0364 - val_loss: 0.0354
Epoch 5/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0332 - val_loss: 0.0300
Epoch 6/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0292 - val_loss: 0.0394
Epoch 7/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0262 - val_loss: 0.0275
Epoch 8/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0254 - val_loss: 0.0239
Epoch 9/50
730/730 [==============================] - 11s 15ms/step - loss: 0.0235 - val_loss: 0.0221
Epoch 10/50
730/730 [==============================] -

D:\anaconda\envs\anomaly_detection\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
730/730 [==============================] - 13s 14ms/step - loss: 0.0975 - val_loss: 0.0335
Epoch 2/50
730/730 [==============================] - 10s 13ms/step - loss: 0.0330 - val_loss: 0.0335
Epoch 3/50
730/730 [==============================] - 10s 14ms/step - loss: 0.0304 - val_loss: 0.0313
Epoch 4/50
730/730 [==============================] - 10s 13ms/step - loss: 0.0288 - val_loss: 0.0268
Epoch 5/50
730/730 [==============================] - 10s 13ms/step - loss: 0.0279 - val_loss: 0.0285
Epoch 6/50
730/730 [==============================] - 10s 13ms/step - loss: 0.0272 - val_loss: 0.0264
Epoch 7/50
730/730 [==============================] - 10s 13ms/step - loss: 0.0269 - val_loss: 0.0316
Epoch 8/50
730/730 [==============================] - 10s 13ms/step - loss: 0.0275 - val_loss: 0.0226
Epoch 9/50
730/730 [==============================] - 10s 13ms/step - loss: 0.0266 - val_loss: 0.0216
Epoch 10/50
730/730 [==============================] - 11s 14ms/step - loss: 0.025

D:\anaconda\envs\anomaly_detection\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/50
730/730 [==============================] - 18s 20ms/step - loss: 0.0426 - val_loss: 0.0139
Epoch 2/50
730/730 [==============================] - 15s 21ms/step - loss: 0.0183 - val_loss: 0.0155
Epoch 3/50
730/730 [==============================] - 16s 21ms/step - loss: 0.0130 - val_loss: 0.0066
Epoch 4/50
730/730 [==============================] - 17s 23ms/step - loss: 0.0090 - val_loss: 0.0069
Epoch 5/50
730/730 [==============================] - 19s 25ms/step - loss: 0.0089 - val_loss: 0.0170
Epoch 6/50
730/730 [==============================] - 18s 25ms/step - loss: 0.0079 - val_loss: 0.0071
Epoch 7/50
730/730 [==============================] - 18s 25ms/step - loss: 0.0072 - val_loss: 0.0046
Epoch 8/50
730/730 [==============================] - 19s 25ms/step - loss: 0.0070 - val_loss: 0.0077
Epoch 9/50
730/730 [==============================] - 18s 25ms/step - loss: 0.0065 - val_loss: 0.0071
Epoch 10/50
730/730 [==============================] - 18s 25ms/step - loss: 0.005

D:\anaconda\envs\anomaly_detection\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
